<a href="https://colab.research.google.com/github/jun789-lee/DataAnalyst/blob/main/agents/gemini_data_analytics/intro_gemini_data_analytics_sdk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Intro to Gemini Data Analytics

| Author |
| --- |
| [Aditya Verma](https://github.com/vermaAstra) |

# Background and Overview
The **Conversational Analytics API** lets you chat with your BigQuery or Looker data anywhere, including embedded Looker dashboards, Slack and other chat apps, or even your own web applications. Your team members can get answers where they need them, when they need them, in the applications they use every day. You can find the [Colab example for HTTP here](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/agents/gemini_data_analytics/intro_gemini_data_analytics_http.ipynb). This is a **Pre-GA** product. See [documentation](https://cloud.google.com/gemini/docs/conversational-analytics-api/overview) for more details.

Please provide feedback to conversational-analytics-api-feedback@google.com

This notebook will help you
1. Setting up the SDK
2. Authenticate to Google Cloud
3. Add data
4. Perform agent operations (create, list, get, delete)
5. Manage conversations (create, list, get, delete)
6. Ask questions with your agent

# Get Started

## API Enablement

**Please fill in the billing_project form field with your own Google Cloud project.  The project must have the following APIs enabled:**
-  [cloudaicompanion API](https://console.cloud.google.com/apis/library/cloudaicompanion.googleapis.com)
-  [Gemini Data Analytics API](https://console.cloud.google.com/apis/library/geminidataanalytics.googleapis.com)
-  [BQ API](https://console.cloud.google.com/marketplace/product/google/bigquery.googleapis.com)
-  [Dataform API](https://console.cloud.google.com/apis/library/dataform.googleapis.com)
- [Vertex AI API](https://console.cloud.google.com/apis/library/aiplatform.googleapis.com)

You may pass in any BigQuery project/dataset/table for which you have read permissions.


## Install the client library

In [ ]:
%pip install google-cloud-geminidataanalytics

# Setup

In [ ]:
# @title Authenticate
from google.colab import auth

auth.authenticate_user()

In [ ]:
# @title Imports
import json as json_lib
import time

import altair as alt
import pandas as pd
import proto
from IPython.display import HTML, display
from google.iam.v1 import iam_policy_pb2, policy_pb2
from google.protobuf import field_mask_pb2
from google.protobuf.json_format import MessageToDict

In [ ]:
# @title Define utilities for viewing responses


def handle_text_response(resp):
    parts = resp.parts
    print("".join(parts))


def display_schema(data):
    fields = data.fields
    df = pd.DataFrame(
        {
            "Column": map(lambda field: field.name, fields),
            "Type": map(lambda field: field.type, fields),
            "Description": map(
                lambda field: getattr(field, "description", "-"), fields
            ),
            "Mode": map(lambda field: field.mode, fields),
        }
    )
    display(df)


def display_section_title(text):
    display(HTML(f"<h2>{text}</h2>"))


def format_looker_table_ref(table_ref):
    return f"lookmlModel: {table_ref.lookml_model}, explore: {table_ref.explore}"


def format_bq_table_ref(table_ref):
    return f"{table_ref.project_id}.{table_ref.dataset_id}.{table_ref.table_id}"


def display_datasource(datasource):
    source_name = ""
    if "studio_datasource_id" in datasource:
        source_name = datasource.studio_datasource_id
    elif "looker_explore_reference" in datasource:
        source_name = format_looker_table_ref(datasource.looker_explore_reference)
    else:
        source_name = format_bq_table_ref(datasource.bigquery_table_reference)

    print(source_name)
    display_schema(datasource.schema)


def handle_schema_response(resp):
    if "query" in resp:
        print(resp.query.question)
    elif "result" in resp:
        display_section_title("Schema resolved")
        print("Data sources:")
        for datasource in resp.result.datasources:
            display_datasource(datasource)


def handle_data_response(resp):
    if "query" in resp:
        query = resp.query
        display_section_title("Retrieval query")
        print(f"Query name: {query.name}")
        print(f"Question: {query.question}")
        print("Data sources:")
        for datasource in query.datasources:
            display_datasource(datasource)
    elif "generated_sql" in resp:
        display_section_title("SQL generated")
        print(resp.generated_sql)
    elif "result" in resp:
        display_section_title("Data retrieved")

        fields = [field.name for field in resp.result.schema.fields]
        d = {}
        for el in resp.result.data:
            for field in fields:
                if field in d:
                    d[field].append(el[field])
                else:
                    d[field] = [el[field]]

        display(pd.DataFrame(d))


def handle_chart_response(resp):
    def _value_to_dict(v):
        if isinstance(v, proto.marshal.collections.maps.MapComposite):
            return _map_to_dict(v)
        if isinstance(v, proto.marshal.collections.RepeatedComposite):
            return [_value_to_dict(el) for el in v]
        if isinstance(v, (int, float, str, bool)):
            return v
        return MessageToDict(v)

    def _map_to_dict(d):
        out = {}
        for k in d:
            if isinstance(d[k], proto.marshal.collections.maps.MapComposite):
                out[k] = _map_to_dict(d[k])
            else:
                out[k] = _value_to_dict(d[k])
        return out

    if "query" in resp:
        print(resp.query.instructions)
    elif "result" in resp:
        vegaConfig = resp.result.vega_config
        vegaConfig_dict = _map_to_dict(vegaConfig)
        alt.Chart.from_json(json_lib.dumps(vegaConfig_dict)).display()


def show_message(msg):
    m = msg.system_message
    if "text" in m:
        handle_text_response(m.text)
    elif "schema" in m:
        handle_schema_response(m.schema)
    elif "data" in m:
        handle_data_response(m.data)
    elif "chart" in m:
        handle_chart_response(m.chart)
    print("\n")

In [ ]:
# @title Import the Gemini Data Analytics client library


from google.cloud import geminidataanalytics_v1beta as geminidataanalytics

data_agent_client = geminidataanalytics.DataAgentServiceClient()
data_chat_client = geminidataanalytics.DataChatServiceClient()

In [ ]:
# @title Billing Project and sample Prompt (System Instruction)

# fmt: off
billing_project = "fm-standupstudio-cpi"  # @param {type:"string"}

location = "global"

# provide critical context for your Conversational Analytics Agent here
system_instruction = "Think like an Analyst"  # @param {type:"string"}
# fmt: on

In [ ]:
# @title BigQuery Datasource

# List datasource(s) you'd like to ask questions of.
# This includes all 12 tables from the SQL KPI Query project.
# Make sure you have permission to query any tables listed here.

# fmt: off

# Table 1: All Events (Raw Data)
bq_project_id_1 = "fm-standupstudio-cpi"  # @param {type:"string"}
bq_dataset_id_1 = "CPI_Evaluation"  # @param {type:"string"}
bq_table_id_1 = "all_events"  # @param {type:"string"}

# Table 2: Average Session Duration
bq_project_id_2 = "fm-standupstudio-cpi"  # @param {type:"string"}
bq_dataset_id_2 = "KPI"  # @param {type:"string"}
bq_table_id_2 = "Average_Session_Duration"  # @param {type:"string"}

# Table 3: Advertisement Count by Ad Type
bq_project_id_3 = "fm-standupstudio-cpi"  # @param {type:"string"}
bq_dataset_id_3 = "KPI"  # @param {type:"string"}
bq_table_id_3 = "Advertise"  # @param {type:"string"}

# Table 4: Daily Active Users (DAU)
bq_project_id_4 = "fm-standupstudio-cpi"  # @param {type:"string"}
bq_dataset_id_4 = "KPI"  # @param {type:"string"}
bq_table_id_4 = "DAU"  # @param {type:"string"}

# Table 5: Difficulty & Balance Indicator
bq_project_id_5 = "fm-standupstudio-cpi"  # @param {type:"string"}
bq_dataset_id_5 = "KPI"  # @param {type:"string"}
bq_table_id_5 = "Difficulty_Balance_Indicator"  # @param {type:"string"}

# Table 6: Early User Progression By Session
bq_project_id_6 = "fm-standupstudio-cpi"  # @param {type:"string"}
bq_dataset_id_6 = "KPI"  # @param {type:"string"}
bq_table_id_6 = "Early_User_Progression_By_Session"  # @param {type:"string"}

# Table 7: eCPM by Title and Time
bq_project_id_7 = "fm-standupstudio-cpi"  # @param {type:"string"}
bq_dataset_id_7 = "KPI"  # @param {type:"string"}
bq_table_id_7 = "eCPM_by_title_and_time"  # @param {type:"string"}

# Table 8: Stage Funnel Indicator
bq_project_id_8 = "fm-standupstudio-cpi"  # @param {type:"string"}
bq_dataset_id_8 = "KPI"  # @param {type:"string"}
bq_table_id_8 = "Stage_Entry_Clear_dropoff_rate"  # @param {type:"string"}

# Table 9: User Feature Matrix for Clustering
bq_project_id_9 = "fm-standupstudio-cpi"  # @param {type:"string"}
bq_dataset_id_9 = "KPI"  # @param {type:"string"}
bq_table_id_9 = "query_build_user_feature_matrix"  # @param {type:"string"}

# Table 10: First Active Retention Rate
bq_project_id_10 = "fm-standupstudio-cpi"  # @param {type:"string"}
bq_dataset_id_10 = "KPI"  # @param {type:"string"}
bq_table_id_10 = "first_active_retention_rate"  # @param {type:"string"}

# Table 11: Revenue Based on eCPM
bq_project_id_11 = "fm-standupstudio-cpi"  # @param {type:"string"}
bq_dataset_id_11 = "KPI"  # @param {type:"string"}
bq_table_id_11 = "Revenue_Based_on_eCPM"  # @param {type:"string"}

# Table 12: Session Per User by Title
bq_project_id_12 = "fm-standupstudio-cpi"  # @param {type:"string"}
bq_dataset_id_12 = "KPI"  # @param {type:"string"}
bq_table_id_12 = "Session_Per_User_by_title"  # @param {type:"string"}

# fmt: on

# Setup - Create table references for all 12 tables
bigquery_table_reference_1 = geminidataanalytics.BigQueryTableReference(
    project_id=bq_project_id_1, dataset_id=bq_dataset_id_1, table_id=bq_table_id_1
)

bigquery_table_reference_2 = geminidataanalytics.BigQueryTableReference(
    project_id=bq_project_id_2, dataset_id=bq_dataset_id_2, table_id=bq_table_id_2
)

bigquery_table_reference_3 = geminidataanalytics.BigQueryTableReference(
    project_id=bq_project_id_3, dataset_id=bq_dataset_id_3, table_id=bq_table_id_3
)

bigquery_table_reference_4 = geminidataanalytics.BigQueryTableReference(
    project_id=bq_project_id_4, dataset_id=bq_dataset_id_4, table_id=bq_table_id_4
)

bigquery_table_reference_5 = geminidataanalytics.BigQueryTableReference(
    project_id=bq_project_id_5, dataset_id=bq_dataset_id_5, table_id=bq_table_id_5
)

bigquery_table_reference_6 = geminidataanalytics.BigQueryTableReference(
    project_id=bq_project_id_6, dataset_id=bq_dataset_id_6, table_id=bq_table_id_6
)

bigquery_table_reference_7 = geminidataanalytics.BigQueryTableReference(
    project_id=bq_project_id_7, dataset_id=bq_dataset_id_7, table_id=bq_table_id_7
)

bigquery_table_reference_8 = geminidataanalytics.BigQueryTableReference(
    project_id=bq_project_id_8, dataset_id=bq_dataset_id_8, table_id=bq_table_id_8
)

bigquery_table_reference_9 = geminidataanalytics.BigQueryTableReference(
    project_id=bq_project_id_9, dataset_id=bq_dataset_id_9, table_id=bq_table_id_9
)

bigquery_table_reference_10 = geminidataanalytics.BigQueryTableReference(
    project_id=bq_project_id_10, dataset_id=bq_dataset_id_10, table_id=bq_table_id_10
)

bigquery_table_reference_11 = geminidataanalytics.BigQueryTableReference(
    project_id=bq_project_id_11, dataset_id=bq_dataset_id_11, table_id=bq_table_id_11
)

bigquery_table_reference_12 = geminidataanalytics.BigQueryTableReference(
    project_id=bq_project_id_12, dataset_id=bq_dataset_id_12, table_id=bq_table_id_12
)

# Optional example queries
example_queries = [
    geminidataanalytics.ExampleQuery(
        natural_language_question="How many total events are in the raw data?",
        sql_query="SELECT COUNT(*) as total_events FROM `fm-standupstudio-cpi.CPI_Evaluation.all_events`",
    ),
    geminidataanalytics.ExampleQuery(
        natural_language_question="What is the average session duration by title?",
        sql_query="SELECT title, AVG(Average_Session_Duration_seconds) as avg_duration FROM `fm-standupstudio-cpi.KPI.Average_Session_Duration` GROUP BY title",
    ),
    geminidataanalytics.ExampleQuery(
        natural_language_question="What is the DAU for each title?",
        sql_query="SELECT title, event_date, DAU FROM `fm-standupstudio-cpi.KPI.DAU` ORDER BY event_date DESC",
    ),
    geminidataanalytics.ExampleQuery(
        natural_language_question="What are the retention rates by title?",
        sql_query="SELECT title, Day1, Day7, Day30 FROM `fm-standupstudio-cpi.KPI.first_active_retention_rate`",
    ),
]

In [ ]:
# @title Looker Datasource

# List up to 5 Looker datasources you'd like to ask questions of.
# This example includes two datasources.

# Looker datasource 1
lookml_model_1 = "<add lookml_model here>"  # @param {type:"string"}
explore_1 = "<add explore here>"  # @param {type:"string"}

# Looker datasource 2 (optional)
# lookml_model_2 = "<add lookml_model here>"  # @param {type:"string"}
# explore_2 = "<add explore here>"  # @param {type:"string"}


# User Credentials (apply to all Looker datasources)
looker_client_id = "<add client_id here>"  # @param {type:"string"}
looker_client_secret = "<add client_secret here>"  # @param {type:"string"}
# OR
looker_access_token = "<add access_token here>"  # @param {type:"string"}


# Required only for Looker PUBLIC instance
looker_instance_uri = "https://my_company.looker.com"  # @param {type:"string"}

# Required only for Looker PRIVATE instance
# looker_instance_id = "<add looker_instance_id here>"  # @param {type:"string"}
# fmt: off
# looker_service_directory_name = "<add looker_service_directory_name here>"  # @param {type:"string"}
# fmt: on


# When connecting to the Looker datasource, you can authenticate using either:
# 1. client_id and client_secret
# 2. access_token
credentials = geminidataanalytics.Credentials(
    oauth=geminidataanalytics.OAuthCredentials(
        secret=geminidataanalytics.OAuthCredentials.SecretBased(
            client_id=looker_client_id, client_secret=looker_client_secret
        ),
        # # If using access_token instead, use this instead and comment out `secret`:
        # token=geminidataanalytics.OAuthCredentials.TokenBased(
        #     access_token = looker_access_token
        # )
    )
)


# Looker datasource setup for PUBLIC Instance
looker_explore_reference_1 = geminidataanalytics.LookerExploreReference(
    looker_instance_uri=looker_instance_uri,
    lookml_model=lookml_model_1,
    explore=explore_1,
)

# looker_explore_reference_2 = geminidataanalytics.LookerExploreReference(
#     looker_instance_uri=looker_instance_uri, lookml_model=lookml_model_2, explore=explore_2
# )

looker_explore_references = [looker_explore_reference_1]
# looker_explore_references.append(looker_explore_reference_2)


# # Looker datasource setup for PRIVATE Instance
# looker_explore_reference = geminidataanalytics.LookerExploreReference(
#     private_looker_instance_info=geminidataanalytics.PrivateLookerInstanceInfo(
#         looker_instance_id=looker_instance_id,
#         service_directory_name=looker_service_directory_name,
#     ),
#     lookml_model=lookml_model_1,
#     explore=explore_1
# )
# credentials = geminidataanalytics.Credentials(
#     oauth=geminidataanalytics.OAuthCredentials(
#         token=geminidataanalytics.OAuthCredentials.TokenBased(
#             access_token = looker_access_token
#         )
#     )
# )

In [ ]:
# @title Looker Studio Datasource

# fmt: off
studio_datasource_id = "<add studio_datasource_id here>"  # @param {type:"string"}
# fmt: on

# Setup
studio_references = geminidataanalytics.StudioDatasourceReference(
    datasource_id=studio_datasource_id
)

In [ ]:
# @title Connect to your Data Source


# Connect to your BigQuery datasource
datasource_references = geminidataanalytics.DatasourceReferences(
    bq=geminidataanalytics.BigQueryTableReferences(
        table_references=[
            bigquery_table_reference_1,   # all_events (Raw Data)
            bigquery_table_reference_2,   # Average_Session_Duration
            bigquery_table_reference_3,   # Advertise
            bigquery_table_reference_4,   # DAU
            bigquery_table_reference_5,   # Difficulty_Balance_Indicator
            bigquery_table_reference_6,   # Early_User_Progression_By_Session
            bigquery_table_reference_7,   # eCPM_by_title_and_time
            bigquery_table_reference_8,   # Stage_Entry_Clear_dropoff_rate
            bigquery_table_reference_9,   # query_build_user_feature_matrix
            bigquery_table_reference_10,  # first_active_retention_rate
            bigquery_table_reference_11,  # Revenue_Based_on_eCPM
            bigquery_table_reference_12,  # Session_Per_User_by_title
        ]
    ),
)

# # Connect to your Looker datasource
# datasource_references = geminidataanalytics.DatasourceReferences(
#     looker=geminidataanalytics.LookerExploreReferences(
#         explore_references=looker_explore_references,
#         # credentials=credentials # Note: uncomment this only in case of stateless chat via inline context
#     ),
# )

# # Connect to your Looker Studio datasource
# datasource_references = geminidataanalytics.DatasourceReferences(
#     studio=geminidataanalytics.StudioDatasourceReferences(
#         studio_references=[studio_references]
#     )
# )


# Context set-up
published_context = geminidataanalytics.Context(
    system_instruction=system_instruction,
    datasource_references=datasource_references,
    example_queries=example_queries,  # Optional - Only supported for BigQuery
    options=geminidataanalytics.ConversationOptions(
        analysis=geminidataanalytics.AnalysisOptions(
            python=geminidataanalytics.AnalysisOptions.Python(
                enabled=True
            )  # if wanting to use advanced analysis with python
        )
    ),
)


# Context set-up for chat using inline context
inline_context = geminidataanalytics.Context(
    system_instruction=system_instruction,
    datasource_references=datasource_references,
    example_queries=example_queries,  # Optional - Only supported for BigQuery
    options=geminidataanalytics.ConversationOptions(
        analysis=geminidataanalytics.AnalysisOptions(
            python=geminidataanalytics.AnalysisOptions.Python(
                enabled=True
            )  # if wanting to use advanced analysis with python
        )
    ),
)

### Write an effective prompt (system_instruction)
- Providing more context around your business and your data improves the quality of answers. We recommend (though do not require) you follow our example `system_instruction` below and fill out the stringified YAML **template** provided with field instructions, validated ("golden") queries, relationships, business terms, etc.
- We are working to improve how our product ingests and uses this context to improve question-answering accuracy

In [ ]:
###############################################################################
#######         Define prompt / system instruction for the question.             #######
#######    It's optional and can guide the agent's response strategy.   #######
###############################################################################

###############################################################################
##### Example template for system instruction for a sales analyst agent:  #####
###############################################################################

# """
# - system_instruction: >-
#     You are an expert sales analyst and understand how to answer questions about
#     the sales data for a fictitious e-commence store
# - tables:
#     - table:
#         - name: bigquery-public-data.thelook_ecommerce.orders
#         - description: orders for The Look fictitious e-commerce store.
#         - synonyms: sales
#         - tags: 'sale, order, sales_order'
#         - fields:
#             - field:
#                 - name: order_id
#                 - description: unique identifier for each order
#             - field:
#                 - name: user_id
#                 - description: unique identifier for each user
#             - field:
#                 - name: status
#                 - description: status of the order
#                 - sample_values:
#                     - complete
#                     - shipped
#                     - returned
#             - field:
#                 - name: gender
#                 - description: gender of the user
#                 - sample_values:
#                     - male
#                     - female
#             - field:
#                 - name: created_at
#                 - description: date and time when the order was created in timestamp format
#             - field:
#                 - name: returned_at
#                 - description: >-
#                     date and time when the order was returned in timestamp
#                     format
#             - field:
#                 - name: num_of_item
#                 - description: number of items in the order
#                 - aggregations: 'sum, avg'
#             - field:
#                 - name: earnings
#                 - description: total sales from the order
#                 - aggregations: 'sum, avg'
#             - field:
#                 - name: cost
#                 - description: total cost to get the items for the order
#                 - aggregations: 'sum, avg'
#         - measures:
#             - measure:
#                 - name: profit
#                 - description: raw profit
#                 - exp: cost - earnings
#                 - synonyms: gains
#         - golden_queries:
#             - golden_query:
#                 - natural_language_query: How many orders are there?
#                 - sql_query: SELECT COUNT(*) FROM sqlgen-testing.thelook_ecommerce.orders
#             - golden_query:
#                 - natural_language_query: How many orders were shipped?
#                 - sql_query: >-
#                     SELECT COUNT(*) FROM sqlgen-testing.thelook_ecommerce.orders
#                     WHERE status = 'shipped'
#         - golden_action_plans:
#             - golden_action_plan:
#                 - natural_language_query: Show me the number of orders broken down by gender.
#                 - action_plan:
#                     - step: >-
#                         Run a SQL query on the table
#                         sqlgen-testing.thelook_ecommerce.orders to get a
#                         breakdown of order count by gender.
#                     - step: >-
#                         Create a vertical bar plot using the retrieved data,
#                         with one bar per gender.
#     - table:
#         - name: sqlgen-testing.thelook_ecommerce.users
#         - description: user of The Look fictitious e-commerce store.
#         - synonyms: customers
#         - tags: 'user, customer, buyer'
#         - fields:
#             - field:
#                 - name: id
#                 - description: unique identifier for each user
#             - field:
#                 - name: first_name
#                 - description: first name of the user
#                 - tag: person
#                 - sample_values: 'graham, adam, brian'
#             - field:
#                 - name: last_name
#                 - description: first name of the user
#                 - tag: person
#                 - sample_values: 'warmer, stilles, smith'
#             - field:
#                 - name: gender
#                 - description: gender of the user
#                 - sample_values:
#                     - male
#                     - female
#             - field:
#                 - name: email
#                 - description: email of the user
#                 - tag: contact
#                 - sample_values: 'brian@gmail.com, graham@gmail.com'
#         - golden_queries:
#             - golden_query:
#                 - natural_language_query: How many unique customers are there?
#                 - sql_query: >-
#                     SELECT COUNT(DISTINCT id) FROM
#                     sqlgen-testing.thelook_ecommerce.users
#             - golden_query:
#                 - natural_language_query: How many female users have gmail email id?
#                 - sql_query: >-
#                     SELECT COUNT(DISTINCT id) FROM
#                     sqlgen-testing.thelook_ecommerce.users WHERE users.gender =
#                     'F' AND users.email LIKE '%@gmail.com';
#     - relationships:
#         - relationship:
#             - name: earnings_to_user
#             - description: >-
#                 Sales table is related to the users table and can be joined for
#                 aggregated view.
#             - relationship_type: many-to-one
#             - join_type: left
#             - left_table: str sqlgen-testing.thelook_ecommerce.orders
#             - right_table: sqlgen-testing.thelook_ecommerce.users
#             - relationship_columns: '// Join columns - left_column:''user_id'' - right_column:''id'''
# - glossaries:
#     - glossary:
#         - term: male
#         - description: male gender
#     - glossary:
#         - term: female
#         - description: female gender
#     - glossary:
#         - term: complete
#         - description: complete status
#         - synonyms: 'finish, done, fulfilled'
#     - glossary:
#         - term: shipped
#         - description: shipped status
#     - glossary:
#         - term: returned
#         - description: returned status
#     - glossary:
#         - term: OMPF
#         - description: Order Management and Product Fulfillment
# - additional_descriptions:
#     - text: All the sales data is for Looker organization.
#     - text: 'Orders can be of three categories, food , clothes, electronics.'
# """

###############################################################################
######    Use the schema below to provide your own system instruction.   ######
###############################################################################

# """
# - system_instruction: str # Expected behavior of the agent. Eg. You are a sales analyst.
# - tables:
#     - table: # Table relevant for this agent.
#         - name: str # Name of the table.
#         - description: str # Description of the table.
#         - synonyms: list[str] # List of synonyms used to refer to the table.
#         - tags: list[str] # List of tags associated with the table.
#         - fields: # Fields in the table.
#             - field:
#             - name: str # Name of the column.
#             - description: str # Description of the column.
#             - synonyms: list[str] # List of synonyms used to refer to the column.
#             - tags: list[str] # List of tags associated with the column.
#             - sample_values: list[str] # List of sample values in the column.
#             - aggregations: list[str] # Any commonly used or default aggregations associated with the column.
#         - measures: # Measures for the table.
#             - measure:
#                 - name: str # Name of the measure.
#                 - description: str # Description of the measure.
#                 - exp: str # Expression to construct the measure.
#                 - synonyms: list[str] # List of synonyms used to refer to the measure.
#         - golden_queries: # Golden or popular queries for the table.
#             - golden_query:
#                 - natural_language_query: str # Natural language query.
#                 - sql_query: str # SQL query.
#         - golden_action_plans: # Golden action plans as the suggested steps to take (in order) to answer the query.
#           - golden_action_plan:
#             - natural_language_query: str # Natural language query.
#             - action_plan:
#               - step: str # Step to take.
#     - relationships: # Join relationships between tables.
#         - relationship:
#           - name: str # Name of the relationship.
#           - description: str # Description of the relationship.
#           - relationship_type: str # Relationship type: one-to-one, one-to-many, many-to-one, many-to-many.
#           - join_type: str # Join type: inner, outer, left, right, full
#           - left_table: str # Left table name.
#           - right_table: str # Right table name.
#           - relationship_columns: # Join columns.
#               - left_column: str # Join column from left table.
#               - right_column: str # Join column from right table.
# - glossaries: # Business glossary, jargon, etc.
#     - glossary:
#         - term: str # Name of the term. Term can be a word, phrase, abbreviation, etc.
#         - description: str # Description or definition of the term.
#         - synonyms: list[str] # List of synonyms for the term.
# - additional_descriptions:
#     - text: str # Any additional description that was not covered above.
# """

In [ ]:
##################### OPTION 1 ###############################
#######                                                #######
#######        Fill the above YAML template for        #######
#######  system_instruction as shown in example above. #######
#######                                                #######
##############################################################

system_instruction = """
###############################################################################
# Game Analytics AI Agent - System Instruction
# Project: GA4 BigQuery KPI Query System (Zombie Session Protected)
# Created: 2025-11-04
###############################################################################

system_instruction: >-
  You are an expert mobile game analytics analyst specializing in GA4 (Google Analytics 4)
  BigQuery data analysis. You understand player behavior metrics, session analytics,
  retention patterns, monetization KPIs, and game balance indicators.

  CRITICAL: All session-related queries in this project apply a 30-minute inactivity
  timeout rule to prevent "zombie session" inflation. When analyzing session data,
  always reference the corrected tables that use sub-session grouping logic.

tables:
  # ========================================================================
  # RAW EVENT DATA SOURCE (BASE TABLE)
  # ========================================================================
  - table:
      name: fm-standupstudio-cpi.CPI_Evaluation.all_events
      description: >-
        Consolidated raw GA4 event data from all analytics datasets (analytics_*).
        This is the foundational table used by all KPI queries.
        WARNING: Direct queries on this table require proper zombie-session protection logic.
      synonyms:
        - raw_events
        - event_data
        - ga4_events
        - source_events
      tags:
        - raw_data
        - events
        - ga4
        - source
      fields:
        - field:
            name: dataset_id
            description: Source analytics dataset name (e.g., 'analytics_123456789')
            tags: [identifier, source]
        - field:
            name: event_date
            description: Date when the event was logged (YYYYMMDD format)
            tags: [date, time]
        - field:
            name: event_timestamp
            description: Event timestamp in microseconds (INT64)
            tags: [timestamp, time]
        - field:
            name: event_name
            description: >-
              Name of the event. 45 event types across 6 categories:
              STAGE PROGRESSION (Stage_Start, Stage_Clear, Stage_Fail, Stage_Restart, Level_Start,
              Level_Clear, Level_Restart, StageFall),
              GAMEPLAY ACTIONS (Add_Move, TapCount, Tap_Count, Make_Invisible, Merge_Count,
              Deal_Count, Add_Time, Miss, Back_Previous),
              ECONOMY (Coin_Acquisition, Coin_Aquisition, Purchase, Life_Refill, Use_Life,
              Use_Refill, Use_Item, Item_Refill),
              ADS (Watch_Banner, Watch_Reward, Watch_Inste, ad_reward, ad_revenue_paid,
              Show_Reward, Revenue_Banner, banner_impression),
              APP LIFECYCLE (first_open, App_Open, session_start, user_engagement, screen_view,
              app_update, app_remove, app_clear_data, os_update, Withdrawal),
              MARKETING (firebase_campaign).
              NOTE: Specific events vary by title; not all titles have all event types.
            tags: [event, action]
            sample_values:
              - Stage_Start
              - Stage_Clear
              - Stage_Fail
              - first_open
              - session_start
              - app_remove
              - app_clear_data
        - field:
            name: event_params
            description: >-
              Array of event parameters (key-value pairs). 50 parameter keys across 9 categories:
              GA4 STANDARD (engaged_session_event, debug_event, campaign, campaign_info_source,
              click_timestamp, content),
              STAGE/LEVEL (Stage, StageName, Level, LevelName, LastClearedStageIndex),
              TIME METRICS (ClearTime, StageTime, LimitTime, Remaining_Time),
              ECONOMY (Coin, Remaining_Coin, Remaining_life, Remaining_spare, Product, Amount),
              GAME MECHANICS (Remaining_Move, TargetMatchNum, TargetPoint, IsDraw),
              ITEMS (Item_Type, Refill, Refill_Method),
              ADS (ad_format, ad_network, ad_unit_code, ad_unit_id, eCPM, ecpm_usd),
              USER ACTIONS (Playing, Trigger, Type, Reward, Reward_Type, Add_Method,
              Quit_Reason, Fail_Type, Reason, Place, CanWatch),
              SESSION IDENTIFIER (ga_session_id - CRITICAL for zombie session fix).
              EXTRACTION EXAMPLE:
              (SELECT value.int_value FROM UNNEST(event_params) WHERE key = 'ga_session_id')
            tags: [parameters, nested, critical]
        - field:
            name: event_previous_timestamp
            description: Timestamp of the previous event in microseconds
            tags: [timestamp, time]
        - field:
            name: user_pseudo_id
            description: Unique pseudo identifier for the user (GA4 client ID)
            tags: [identifier, user]
        - field:
            name: user_id
            description: User ID set via setUserId API (if available)
            tags: [identifier, user]
        - field:
            name: user_properties
            description: >-
              Array of user properties. Two main properties:
              1) first_open_time (INT64): User's first app open timestamp in MILLISECONDS,
              DATE-ROUNDED (loses time-of-day precision, e.g., 1760306400000 = 2025-10-12 00:00:00).
              2) last_gclid (STRING): Most recent Google Ads click ID for attribution.
              EXTRACTION EXAMPLE:
              (SELECT value.int_value FROM UNNEST(user_properties) WHERE key = 'first_open_time')
              CONVERSION: TIMESTAMP_MILLIS(first_open_time) for date analysis.
            tags: [properties, user, nested]
        - field:
            name: user_first_touch_timestamp
            description: >-
              Timestamp when user was first seen (MICROSECONDS, INT64).
              ROUNDED to ~150ms intervals (e.g., 1760304123150000), loses sub-second precision.
              UNLIKE user_properties.first_open_time (date-rounded), this retains time-of-day
              but loses millisecond precision. Use TIMESTAMP_MICROS() for conversion.
              RECOMMENDATION: Use this for time-of-day analysis; use first_open_time for date cohorts.
            tags: [timestamp, acquisition, rounded]
        - field:
            name: privacy_info
            description: >-
              Struct containing user privacy consent status (GDPR/CCPA compliance).
              Fields: analytics_storage (YES/NO/UNSET), ads_storage (YES/NO/UNSET),
              uses_transient_token (0/1/3 for token usage).
              IMPACT: NO/UNSET consent may reduce data availability. Regional variations apply.
              EXTRACTION EXAMPLE:
              privacy_info.analytics_storage, privacy_info.ads_storage
            tags: [privacy, consent, gdpr, ccpa, nested]
        - field:
            name: device
            description: Device information (category, mobile brand name, model, OS, etc.)
            tags: [device, nested]
        - field:
            name: geo
            description: Geographic information (country, region, city, etc.)
            tags: [geo, location, nested]
        - field:
            name: app_info
            description: App information (id, version, install_source, etc.)
            tags: [app, nested]
        - field:
            name: platform
            description: Platform where the app is running (ANDROID, IOS, WEB)
            tags: [platform, device]
        - field:
            name: stream_id
            description: Analytics data stream ID
            tags: [identifier, stream]
        - field:
            name: traffic_source
            description: Traffic source information (source, medium, campaign)
            tags: [attribution, source, nested]
      golden_queries:
        - golden_query:
            natural_language_query: How many Stage_Start events occurred yesterday?
            sql_query: >-
              SELECT COUNT(*)
              FROM `fm-standupstudio-cpi.CPI_Evaluation.all_events`
              WHERE event_date = FORMAT_DATE('%Y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
                AND event_name = 'Stage_Start'
        - golden_query:
            natural_language_query: What are the most common event names?
            sql_query: >-
              SELECT event_name, COUNT(*) AS event_count
              FROM `fm-standupstudio-cpi.CPI_Evaluation.all_events`
              WHERE event_date >= FORMAT_DATE('%Y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY))
              GROUP BY event_name
              ORDER BY event_count DESC
              LIMIT 10
        - golden_query:
            natural_language_query: How many unique users were active last week?
            sql_query: >-
              SELECT COUNT(DISTINCT user_pseudo_id) AS active_users
              FROM `fm-standupstudio-cpi.CPI_Evaluation.all_events`
              WHERE event_date >= FORMAT_DATE('%Y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY))
      golden_action_plans:
        - golden_action_plan:
            natural_language_query: Calculate session duration from raw events
            action_plan:
              - step: >-
                  Extract ga_session_id from event_params array using
                  UNNEST and WHERE key = 'ga_session_id'
              - step: >-
                  Use LAG() window function to calculate time gaps between consecutive events
                  within the same user_pseudo_id and ga_session_id
              - step: >-
                  Identify breaks where time gap > 1800 seconds (30 minutes) and flag as new sub-sessions
              - step: >-
                  Assign sub_session_group IDs using cumulative SUM() of break flags
              - step: >-
                  Filter only sub_session_group = 1 and calculate MIN/MAX timestamp difference

  # ========================================================================
  # CORE USER ANALYTICS TABLE
  # ========================================================================
  - table:
      name: fm-standupstudio-cpi.KPI.query_build_user_feature_matrix
      description: >-
        Comprehensive user-level feature matrix for clustering and behavioral analysis.
        Contains aggregated session, event, and engagement data per user per game title.
        ZOMBIE-SESSION PROTECTED: Uses 30-minute inactivity timeout for accurate playtime.
      synonyms:
        - user_features
        - user_matrix
        - player_features
        - clustering_data
      tags:
        - user
        - session
        - playtime
        - clustering
        - engagement
      fields:
        - field:
            name: user_pseudo_id
            description: Unique identifier for each user (GA4 pseudo ID)
            tags: [identifier, user]
        - field:
            name: title
            description: Game title and version in format 'game_id-version'
            tags: [game, title]
            sample_values:
              - colorfuldice-0.52
              - colorfuldice-0.4
        - field:
            name: total_playtime_seconds
            description: Total playtime across all sessions (zombie-session protected with 30-min timeout)
            aggregations: [sum, avg, median, percentile]
            tags: [session, time, engagement]
        - field:
            name: avg_session_duration_sec
            description: Average session duration per user (zombie-session protected)
            aggregations: [avg, median, percentile]
            tags: [session, time]
        - field:
            name: total_session_count
            description: Total number of valid sessions (with 30-min timeout validation)
            aggregations: [sum, avg, count]
            tags: [session, engagement]
        - field:
            name: unique_play_days
            description: Number of unique days user played the game
            aggregations: [avg, median, sum]
            tags: [retention, engagement]
        - field:
            name: days_since_last_play
            description: Days elapsed since user's last activity
            aggregations: [avg, median]
            tags: [retention, churn]
        - field:
            name: user_game_lifespan_days
            description: Total days from first to last play
            aggregations: [avg, median, percentile]
            tags: [retention, lifetime]
        - field:
            name: count_event_Stage_Start
            description: Total number of stage start events
            aggregations: [sum, avg]
            tags: [progression, engagement]
        - field:
            name: count_event_Stage_Clear
            description: Total number of stage clear events
            aggregations: [sum, avg]
            tags: [progression, success]
        - field:
            name: max_Stage_reached
            description: Highest stage number reached by user
            aggregations: [max, avg, median]
            tags: [progression, skill]
        - field:
            name: clear_to_start_ratio
            description: Ratio of stage clears to stage starts (completion efficiency)
            aggregations: [avg, median]
            tags: [skill, difficulty, success]
        - field:
            name: playtime_per_clear
            description: Average seconds spent per stage clear
            aggregations: [avg, median]
            tags: [efficiency, difficulty]
        - field:
            name: sessions_per_clear
            description: Average sessions needed per stage clear
            aggregations: [avg, median]
            tags: [efficiency, engagement]
      measures:
        - measure:
            name: active_user_rate
            description: Percentage of users active in last 7 days
            exp: COUNT(IF(days_since_last_play <= 7, user_pseudo_id, NULL)) / COUNT(user_pseudo_id) * 100
            synonyms: [activity_rate, active_rate]
        - measure:
            name: high_engagement_users
            description: Users with above-median playtime
            exp: COUNT(IF(total_playtime_seconds > MEDIAN(total_playtime_seconds), user_pseudo_id, NULL))
            synonyms: [engaged_users, power_users]
      golden_queries:
        - golden_query:
            natural_language_query: What's the average total playtime per user by title?
            sql_query: >-
              SELECT title, ROUND(AVG(total_playtime_seconds), 2) AS avg_playtime_sec
              FROM `fm-standupstudio-cpi.KPI.query_build_user_feature_matrix`
              GROUP BY title ORDER BY title
        - golden_query:
            natural_language_query: How many users have played more than 1 hour total?
            sql_query: >-
              SELECT title, COUNT(DISTINCT user_pseudo_id) AS users_over_1hr
              FROM `fm-standupstudio-cpi.KPI.query_build_user_feature_matrix`
              WHERE total_playtime_seconds > 3600
              GROUP BY title
        - golden_query:
            natural_language_query: What's the median stage reached by users?
            sql_query: >-
              SELECT title,
                PERCENTILE_CONT(max_Stage_reached, 0.5) OVER(PARTITION BY title) AS median_stage
              FROM `fm-standupstudio-cpi.KPI.query_build_user_feature_matrix`
              GROUP BY title

  # ========================================================================
  # SESSION METRICS TABLE
  # ========================================================================
  - table:
      name: fm-standupstudio-cpi.KPI.Average_Session_Duration
      description: >-
        Session duration statistics by game title using middle 95% percentile filtering.
        ZOMBIE-SESSION PROTECTED: Applied 30-minute inactivity timeout (Fixed 2025-11-04).
      synonyms: [session_duration, session_stats, session_length]
      tags: [session, time, statistics]
      fields:
        - field:
            name: title
            description: Game title and version
            tags: [game, identifier]
        - field:
            name: avg_session_length_seconds
            description: Average session duration (middle 95% data, outliers removed)
            aggregations: [avg]
            tags: [session, time, kpi]
        - field:
            name: var_session_length_seconds
            description: Variance of session duration
            tags: [statistics, variance]
        - field:
            name: stddev_session_length_seconds
            description: Standard deviation of session duration
            tags: [statistics, spread]
        - field:
            name: median_session_sec_50pct
            description: Median session duration (50th percentile)
            tags: [session, median, percentile]
        - field:
            name: session_sample_count
            description: Number of sessions included in calculation
            tags: [sample, count]
      golden_queries:
        - golden_query:
            natural_language_query: Which title has the longest average session?
            sql_query: >-
              SELECT title, avg_session_length_seconds
              FROM `fm-standupstudio-cpi.KPI.Average_Session_Duration`
              ORDER BY avg_session_length_seconds DESC LIMIT 1

  # ========================================================================
  # DIFFICULTY & BALANCE TABLE
  # ========================================================================
  - table:
      name: fm-standupstudio-cpi.KPI.Difficulty_Balance_Indicator
      description: >-
        Comprehensive stage difficulty metrics including attempts, clear times, dropout rates.
        Separates statistics for clearing vs. dropping-off users.
        ZOMBIE-SESSION PROTECTED: Session dwell time uses 30-min timeout (Fixed 2025-11-04).
      synonyms: [difficulty, balance, stage_metrics, game_balance]
      tags: [difficulty, balance, stage, progression]
      fields:
        - field:
            name: title
            description: Game title and version
            tags: [game, identifier]
        - field:
            name: stage
            description: Stage/level number
            tags: [stage, level, progression]
        - field:
            name: avg_attempts_to_clear
            description: Average number of attempts to clear for successful users
            aggregations: [avg]
            tags: [difficulty, attempts, success]
        - field:
            name: avg_attempts_dropoff
            description: Average number of attempts before dropping off
            aggregations: [avg]
            tags: [difficulty, attempts, churn]
        - field:
            name: avg_clear_time_sec
            description: Average time to clear stage in seconds
            aggregations: [avg, median]
            tags: [time, difficulty, clear]
        - field:
            name: give_up_rate_percent
            description: Percentage of attempts that ended without clearing
            tags: [dropout, churn, difficulty]
        - field:
            name: avg_dwell_clearers_sec
            description: Average session duration for sessions with stage clear
            tags: [session, time, success]
        - field:
            name: avg_dwell_dropoffs_sec
            description: Average session duration for dropout sessions
            tags: [session, time, churn]
      measures:
        - measure:
            name: difficulty_score
            description: Combined difficulty metric based on attempts and dropout rate
            exp: (avg_attempts_to_clear * 0.6) + (give_up_rate_percent * 0.4)
            synonyms: [hardness, challenge_level]
      golden_queries:
        - golden_query:
            natural_language_query: Which stages have the highest dropout rate?
            sql_query: >-
              SELECT title, stage, give_up_rate_percent
              FROM `fm-standupstudio-cpi.KPI.Difficulty_Balance_Indicator`
              WHERE stage <= 30
              ORDER BY give_up_rate_percent DESC LIMIT 10
        - golden_query:
            natural_language_query: What's the average attempts needed for stage 10?
            sql_query: >-
              SELECT title, avg_attempts_to_clear
              FROM `fm-standupstudio-cpi.KPI.Difficulty_Balance_Indicator`
              WHERE stage = 10

  # ========================================================================
  # RETENTION & ENGAGEMENT TABLES
  # ========================================================================
  - table:
      name: fm-standupstudio-cpi.KPI.DAU
      description: >-
        Daily Active Users with release date tracking and cumulative user counts.
      synonyms: [daily_active, dau, active_users]
      tags: [engagement, retention, daily, users]
      fields:
        - field:
            name: date
            description: Event date
            tags: [date, time]
        - field:
            name: dau
            description: Number of unique active users on this date
            aggregations: [sum, avg]
            tags: [users, active, kpi]
        - field:
            name: days_since_release
            description: Number of days elapsed since release
            tags: [time, release]
        - field:
            name: total_unique_users_to_date
            description: Cumulative count of all unique users up to this date
            aggregations: [max]
            tags: [users, cumulative]

  - table:
      name: fm-standupstudio-cpi.KPI.first_active_retention_rate
      description: >-
        Cohort-based retention rates for Days 1-7, 14, and 30 after first active date.
      synonyms: [retention, retention_rate, user_retention]
      tags: [retention, cohort, engagement]
      fields:
        - field:
            name: title
            description: Game title and version
            tags: [game, identifier]
        - field:
            name: total_users
            description: Total number of users in cohort
            tags: [users, cohort]
        - field:
            name: day_1_retention
            description: Percentage of users who returned on Day 1
            tags: [retention, d1, kpi]
        - field:
            name: day_7_retention
            description: Percentage of users who returned on Day 7
            tags: [retention, d7, kpi]
        - field:
            name: day_30_retention
            description: Percentage of users who returned on Day 30
            tags: [retention, d30, kpi]
      golden_queries:
        - golden_query:
            natural_language_query: What's the Day 1 retention rate by title?
            sql_query: >-
              SELECT title, day_1_retention * 100 AS d1_retention_percent
              FROM `fm-standupstudio-cpi.KPI.first_active_retention_rate`
              ORDER BY day_1_retention DESC

  - table:
      name: fm-standupstudio-cpi.KPI.Early_User_Progression_By_Session
      description: >-
        Session 1-10 progression metrics including dwell time, stage reached, and retention.
        ZOMBIE-SESSION PROTECTED: Session dwell time uses 30-min timeout (Fixed 2025-11-04).
      synonyms: [early_progression, session_progression, onboarding]
      tags: [progression, session, early_user, retention]
      fields:
        - field:
            name: session_number
            description: Session sequence number (1-10)
            tags: [session, sequence]
        - field:
            name: avg_session_dwell_time_sec
            description: Average time spent in this session number
            aggregations: [avg]
            tags: [session, time]
        - field:
            name: avg_max_stage_reached
            description: Average highest stage reached in this session
            aggregations: [avg]
            tags: [progression, stage]
        - field:
            name: retention_s1_to_s2_percent
            description: Percentage of S1 users who returned for S2
            tags: [retention, session]

  - table:
      name: fm-standupstudio-cpi.KPI.Session_Per_User_by_title
      description: >-
        Average sessions per user by cohort day (days since acquisition).
        ZOMBIE-SESSION PROTECTED: Uses 30-min timeout for accurate session counting (Fixed 2025-11-04).
      synonyms: [session_per_user, cohort_sessions, daily_sessions]
      tags: [session, cohort, engagement]
      fields:
        - field:
            name: days_since_acquisition
            description: Number of days since user's first event (cohort day)
            tags: [cohort, time]
        - field:
            name: total_sessions
            description: Total number of sessions on this cohort day
            aggregations: [sum]
            tags: [session, count]
        - field:
            name: active_users
            description: Number of users active on this cohort day
            aggregations: [count]
            tags: [users, active]
        - field:
            name: avg_sessions_per_user
            description: Average sessions per active user on this cohort day
            aggregations: [avg]
            tags: [session, kpi, engagement]

  # ========================================================================
  # MONETIZATION TABLES
  # ========================================================================
  - table:
      name: fm-standupstudio-cpi.KPI.Advertise
      description: >-
        Advertisement event analysis by type with outlier handling (top 5% trimming).
      synonyms: [ads, advertisement, ad_events]
      tags: [monetization, ads, revenue]
      fields:
        - field:
            name: Watch_Banner_count
            description: Total number of banner ad views
            aggregations: [sum, avg]
            tags: [ads, banner]
        - field:
            name: Watch_Inste_count
            description: Total number of interstitial ad views
            aggregations: [sum, avg]
            tags: [ads, interstitial]
        - field:
            name: Watch_Reward_count
            description: Total number of rewarded ad views
            aggregations: [sum, avg]
            tags: [ads, reward]
        - field:
            name: total_ad_events
            description: Total number of all advertisement events
            aggregations: [sum, avg]
            tags: [ads, total]
        - field:
            name: avg_ad_events_per_user
            description: Average ad views per user
            aggregations: [avg]
            tags: [ads, users, engagement]

  - table:
      name: fm-standupstudio-cpi.KPI.eCPM_by_title_and_time
      description: >-
        Daily eCPM (effective Cost Per Mille) trends by ad type with cumulative tracking.
      synonyms: [ecpm, ad_revenue, cpm]
      tags: [monetization, ecpm, revenue, ads]
      fields:
        - field:
            name: event_date
            description: Date of the ad events
            tags: [date, time]
        - field:
            name: overall_avg_ecpm
            description: Average eCPM for all ad types combined
            aggregations: [avg]
            tags: [ecpm, revenue, kpi]
        - field:
            name: banner_avg_ecpm
            description: Average eCPM for banner ads
            tags: [ecpm, banner]
        - field:
            name: reward_avg_ecpm
            description: Average eCPM for rewarded ads
            tags: [ecpm, reward]

  - table:
      name: fm-standupstudio-cpi.KPI.Revenue_Based_on_eCPM
      description: >-
        Total revenue and event counts by ad type.
      synonyms: [revenue, ad_revenue, earnings]
      tags: [monetization, revenue, ads]
      fields:
        - field:
            name: total_revenue
            description: Total revenue from all ad types combined
            aggregations: [sum]
            tags: [revenue, total, kpi]
        - field:
            name: banner_revenue
            description: Total revenue from banner ads
            aggregations: [sum]
            tags: [revenue, banner]
        - field:
            name: reward_revenue
            description: Total revenue from rewarded ads
            aggregations: [sum]
            tags: [revenue, reward]

  # ========================================================================
  # PROGRESSION & FUNNEL TABLE
  # ========================================================================
  - table:
      name: fm-standupstudio-cpi.KPI.Stage_Entry_Clear_dropoff_rate
      description: >-
        Stage progression funnel with entry rates, clear rates, and dropoff rates.
      synonyms: [funnel, stage_funnel, progression_funnel]
      tags: [funnel, progression, stage]
      fields:
        - field:
            name: total_reached_users
            description: Number of unique users who reached this stage
            aggregations: [count]
            tags: [users, progression]
        - field:
            name: total_cleared_users
            description: Number of unique users who cleared this stage
            aggregations: [count]
            tags: [users, success]
        - field:
            name: clear_rate_percent
            description: Percentage of users who cleared after reaching
            tags: [rate, success, kpi]
        - field:
            name: StageDropOffRate
            description: Percentage who reached but did not clear (within-stage dropout)
            tags: [dropout, churn, kpi]
        - field:
            name: FunnelRetentionRate
            description: Percentage of stage 1 users who reached this stage
            tags: [retention, funnel, progression]

# ========================================================================
# TABLE RELATIONSHIPS
# ========================================================================
relationships:
  - relationship:
      name: user_to_session_duration
      description: >-
        User feature matrix contains aggregated session data that can be
        cross-referenced with session duration statistics for deeper analysis.
      relationship_type: many-to-one
      join_type: inner
      left_table: fm-standupstudio-cpi.KPI.query_build_user_feature_matrix
      right_table: fm-standupstudio-cpi.KPI.Average_Session_Duration
      relationship_columns:
        left_column: title
        right_column: title

  - relationship:
      name: user_to_retention
      description: >-
        User features can be joined with retention metrics to analyze
        behavioral patterns of retained vs. churned users.
      relationship_type: many-to-one
      join_type: inner
      left_table: fm-standupstudio-cpi.KPI.query_build_user_feature_matrix
      right_table: fm-standupstudio-cpi.KPI.first_active_retention_rate
      relationship_columns:
        left_column: title
        right_column: title

  - relationship:
      name: stage_to_difficulty
      description: >-
        Stage funnel metrics can be joined with difficulty indicators
        to understand which stages cause the most friction.
      relationship_type: one-to-one
      join_type: inner
      left_table: fm-standupstudio-cpi.KPI.Stage_Entry_Clear_dropoff_rate
      right_table: fm-standupstudio-cpi.KPI.Difficulty_Balance_Indicator
      relationship_columns:
        left_column: title, stage
        right_column: title, stage

# ========================================================================
# BUSINESS GLOSSARY
# ========================================================================
glossaries:
  - glossary:
      term: zombie session
      description: >-
        Inflated session duration caused by delayed events (e.g., app_remove)
        occurring days/months after actual gameplay, sharing the same ga_session_id.
        Fixed by applying 30-minute inactivity timeout rule.
      synonyms: [inflated session, ghost session]

  - glossary:
      term: 30-minute inactivity timeout
      description: >-
        Defensive logic that identifies activity gaps > 1800 seconds (30 minutes)
        within a session and only uses the first continuous activity cluster for
        accurate session duration calculation.
      synonyms: [30-min timeout, session timeout, inactivity rule]

  - glossary:
      term: sub_session_group
      description: >-
        Unique ID assigned to continuous activity clusters within a ga_session_id.
        Only sub_session_group = 1 (first cluster) is used for accurate metrics.
      synonyms: [activity cluster, session group]

  - glossary:
      term: middle 95%
      description: >-
        Outlier removal technique that excludes bottom 2.5% and top 2.5% of data,
        keeping only P2.5 to P97.5 for statistical calculations.
      synonyms: [trimmed mean, 95% percentile filter]

  - glossary:
      term: DAU
      description: Daily Active Users - unique users who had at least one event on a given day
      synonyms: [daily active, active users]

  - glossary:
      term: D1 retention
      description: Day 1 retention rate - percentage of users who return on the day after first play
      synonyms: [day 1 retention, next day retention]

  - glossary:
      term: D7 retention
      description: Day 7 retention rate - percentage of users who return 7 days after first play
      synonyms: [day 7 retention, week 1 retention]

  - glossary:
      term: eCPM
      description: >-
        Effective Cost Per Mille (thousand impressions) - revenue generated per 1000 ad views
      synonyms: [effective cpm, cpm]

  - glossary:
      term: stage dropout rate
      description: >-
        Percentage of users who reach a stage but fail to clear it (within-stage churn)
      synonyms: [give up rate, failure rate]

  - glossary:
      term: cohort day
      description: >-
        Number of days since user's first active date (Day 0 = first play, Day 1 = next day, etc.)
      synonyms: [days since acquisition, user age]

  - glossary:
      term: ga_session_id
      description: >-
        Google Analytics session identifier that may be reused across different time periods.
        Extracted from event_params array. CRITICAL for zombie session protection.
      synonyms: [session id, ga session]

  - glossary:
      term: event_params
      description: >-
        Nested array field containing event-specific parameters as key-value pairs.
        Requires UNNEST() to extract values. Contains 50+ parameter types across 9 categories.
      synonyms: [event parameters, params]

  - glossary:
      term: user_properties
      description: >-
        Nested array field containing user-scoped custom properties.
        Includes first_open_time (date-rounded) and last_gclid (attribution).
      synonyms: [user props, custom properties]

  - glossary:
      term: privacy_info
      description: >-
        Struct containing GDPR/CCPA consent status for analytics and ads storage.
        Values: YES (granted), NO (denied), UNSET (not configured).
      synonyms: [consent status, privacy consent]

  - glossary:
      term: timestamp rounding
      description: >-
        GA4 timestamps are rounded for privacy: user_first_touch_timestamp (~150ms intervals),
        first_open_time (date-level), both lose precision. Account for this in time-sensitive analysis.
      synonyms: [time precision loss, timestamp precision]

# ========================================================================
# ADDITIONAL DESCRIPTIONS
# ========================================================================
additional_descriptions:
  - text: >-
      All session-related queries in this system have been updated (2025-11-04) to
      implement the 30-minute inactivity timeout rule, resolving the zombie session problem.

  - text: >-
      Data source is fm-standupstudio-cpi.CPI_Evaluation.all_events, which aggregates
      GA4 event data from multiple analytics datasets since 2024-01-01.

  - text: >-
      When analyzing session metrics, always prioritize tables marked as "ZOMBIE-SESSION PROTECTED"
      for accurate playtime and engagement calculations.

  - text: >-
      For statistical robustness, most KPI tables apply middle 95% filtering (P2.5-P97.5)
      to remove extreme outliers while calculating means and standard deviations.

  - text: >-
      The project focuses on mobile puzzle games with stage-based progression,
      where stages are numbered sequentially and users attempt to clear them.

  - text: >-
      Revenue data is based on in-game advertisements (Banner, Interstitial, Rewarded)
      with eCPM values tracked per ad view event.

  - text: >-
      event_params contains 50 parameter keys across 9 categories (GA4 Standard, Stage/Level,
      Time Metrics, Economy, Game Mechanics, Items, Ads, User Actions, Session Identifier).
      Extract using UNNEST: (SELECT value.int_value FROM UNNEST(event_params) WHERE key = 'param_name')

  - text: >-
      user_properties.first_open_time is DATE-ROUNDED (milliseconds precision, loses time-of-day).
      user_first_touch_timestamp is ROUNDED to ~150ms intervals (microseconds, retains time-of-day).
      Use first_open_time for date cohorts, user_first_touch_timestamp for time-of-day analysis.

  - text: >-
      privacy_info tracks GDPR/CCPA consent (analytics_storage, ads_storage, uses_transient_token).
      Users with NO/UNSET consent may have limited data availability. Check consent status when
      analyzing attribution or personalization features.

  - text: >-
      event_name includes 45 event types across 6 categories (Stage Progression, Gameplay Actions,
      Economy, Ads, App Lifecycle, Marketing). Not all titles have all event types.
      app_remove and app_clear_data are key zombie session triggers.
"""

In [ ]:
##################### OPTION 2 ###############################
#######                                                #######
#######         Leave system_instruction empty         #######
#######         or provide description in form         #######
#######                 of long string.                #######
#######                                                #######
##############################################################

# Define context for question.
# system_instruction is optional, can be used to steer the agent's answering
# strategy.
# fmt: off
system_instruction = "Think like an Analyst"  # @param {type:"string"}
# fmt: on

### *Helper functions for processing the system_instruction yaml string*

In [ ]:
import re
from collections import Counter

import yaml
from yaml.scanner import ScannerError

def cleanup_path(input_string: str) -> str:
    """Removes bracketed numbers and preceding dots from an input string.

    Args:
      input_string: The input string.

    Returns:
      The string with bracketed numbers and preceding dots removed.
    """
    cleaned_string = re.sub(r"\[\d+]", "", input_string)
    if cleaned_string.startswith("."):
        cleaned_string = cleaned_string[1:]
    return cleaned_string


def is_valid_yaml(yaml_string: str) -> bool:
    try:
        yaml.safe_load(yaml_string)
        return True
    except ScannerError:
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False


def print_counter_desc(data: list[str]) -> None:
    # Count the occurrences of each item in the list.
    item_counts = Counter(data)

    # Sort the items in descending order based on their counts.
    sorted_items = sorted(item_counts.items(), key=lambda item: item[1], reverse=True)

    # Print the items and their counts.
    for item, count in sorted_items:
        print(f"{item}: {count}")


def extract_insights_from_system_instruction(yaml_string: str) -> None:
    """Extracts insights into used fields from a YAML string.

    Args:
      yaml_string: The YAML string to analyze.
    """
    if not is_valid_yaml(yaml_string):
        print("Received Invalid or not a YAML format. Cannot extract insights.")
        return

    try:
        data = yaml.safe_load(yaml_string)
        used_fields = []

        def traverse(node, path=""):
            if isinstance(node, dict):
                for key, value in node.items():
                    new_path = f"{path}.{key}" if path else key
                    traverse(value, new_path)
            elif isinstance(node, list):
                for i, item in enumerate(node):
                    new_path = f"{path}[{i}]"
                    traverse(item, new_path)
            else:
                used_fields.append(cleanup_path(path))

        traverse(data)
        print("Insights:")
        print("Number of unique fields used: ", len(set(used_fields)))
        print("Used fields in YAML:")
        print()
        print_counter_desc(used_fields)

    except ScannerError as e:
        print(f"Invalid YAML: {e}")
        return
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return


# Printing insights assuming 'system_instruction' is defined above.
extract_insights_from_system_instruction(system_instruction)

# Example API call
**All API calls** take `billing_project` (a Google Cloud billing project), a `question` (first in a list of Messages), and `prompt` / `system_instruction` (optional)


**BigQuery calls** create a BigQueryTableReference contain: via `project_id`, `dataset_id`, and `table_id`. Note that multiple table references can be passed in a list.


**Looker calls** create a LookerExploreReference with `looker_instance_uri` for public instance or `private_looker_instance_info` for private instance, `lookml_model`, and `explore`. Auth credentials are created with `client_id` and `client_secret` or `access_token`.

*Note: options for advanced system instructions are in the cells below*

In [ ]:
# @title Create Data Agent

# fmt: off
data_agent_id = "data_agent_8"  # @param {type:"string"}
# fmt: on

data_agent = geminidataanalytics.DataAgent(
    data_analytics_agent=geminidataanalytics.DataAnalyticsAgent(
        published_context=published_context
    ),
)

request = geminidataanalytics.CreateDataAgentRequest(
    parent=f"projects/{billing_project}/locations/{location}",
    data_agent_id=data_agent_id,  # Optional
    data_agent=data_agent,
)

try:
    operation = data_agent_client.create_data_agent(request=request)

    while not operation.done():
        print("Still in progress...")
        time.sleep(2)

    if operation.done():
        if operation.exception():
            print("Operation failed:", operation.exception())
        else:
            print("Operation succeeded:", operation.result())
except Exception as e:
    print(f"Error creating Data Agent: {e}")

In [ ]:
# @title Create Conversation

# Initialize request argument(s)
# fmt: off
data_agent_id = "data_agent_8"  # @param {type:"string"}
conversation_id = "conversation_8"  # @param {type:"string"}
# fmt: on

conversation = geminidataanalytics.Conversation(
    agents=[data_chat_client.data_agent_path(billing_project, location, data_agent_id)],
)

request = geminidataanalytics.CreateConversationRequest(
    parent=f"projects/{billing_project}/locations/{location}",
    conversation_id=conversation_id,
    conversation=conversation,
)

# Make the request
response = data_chat_client.create_conversation(request=request)

# Handle the response
print(response)

# Chat request
1. Chat request with Conversation reference
2. Chat request Data Agent
3. Chat request with Inline Context

## [Stateful] Chat request with conversation reference

In [ ]:
# Create a request containing a single user message -- your question.
# fmt: off
question = "Make a bar graph for the top 5 states by the total number of airports"  # @param {type:"string"}
# fmt: on

# Create the user message
messages = [
    geminidataanalytics.Message(
        user_message=geminidataanalytics.UserMessage(text=question)
    )
]

data_agent_id = "data_agent_7"  # @param {type:"string"}
conversation_id = "conversation_7"  # @param {type:"string"}

# Create a conversation_reference
conversation_reference = geminidataanalytics.ConversationReference(
    conversation=data_chat_client.conversation_path(
        billing_project, location, conversation_id
    ),
    data_agent_context=geminidataanalytics.DataAgentContext(
        data_agent=data_chat_client.data_agent_path(
            billing_project, location, data_agent_id
        ),
        # credentials=credentials  # Uncomment if using Looker datasource
    ),
)


# Form the request
request = geminidataanalytics.ChatRequest(
    parent=f"projects/{billing_project}/locations/{location}",
    messages=messages,
    conversation_reference=conversation_reference,
)

# Make the request
stream = data_chat_client.chat(request=request)

# Handle the response
for response in stream:
    show_message(response)

## [Stateless] Chat request with data_agent

In [ ]:
# Create a request containing a single user message -- your question.
# fmt: off
question = "Make a bar graph for the top 5 states by the total number of airports"  # @param {type:"string"}
# fmt: on

# Create the message
messages = [
    geminidataanalytics.Message(
        user_message=geminidataanalytics.UserMessage(text=question)
    )
]

data_agent_id = "data_agent_1"  # @param {type:"string"}

# Create the data agent context
data_agent_context = geminidataanalytics.DataAgentContext(
    data_agent=data_chat_client.data_agent_path(
        billing_project, location, data_agent_id
    ),
    # credentials=credentials  # Uncomment this if you're using Looker datasource
)

# Form the request
request = geminidataanalytics.ChatRequest(
    parent=f"projects/{billing_project}/locations/{location}",
    messages=messages,
    data_agent_context=data_agent_context,
)

# Make the request
stream = data_chat_client.chat(request=request)

# Handle the response
for response in stream:
    show_message(response)

## [Stateless] Chat request with Inline Context

In [ ]:
# Create a request containing a single user message -- your question.
# fmt: off
question = "Make a bar graph for the top 5 states by the total number of airports"  # @param {type:"string"}
# fmt: on

messages = [
    geminidataanalytics.Message(
        user_message=geminidataanalytics.UserMessage(text=question)
    )
]

request = geminidataanalytics.ChatRequest(
    inline_context=inline_context,
    parent=f"projects/{billing_project}/locations/{location}",
    messages=messages,
)

# Make the request
stream = data_chat_client.chat(request=request)

# Handle the response
for response in stream:
    show_message(response)

# Multi-turn Conversation with Data Agent or Inline Context

In [ ]:
# @title Multi-turn Conversation

# Re-used across requests to track previous turns
conversation_messages = []

# fmt: off
data_agent_id = "data_agent_8"  # @param {type:"string"}
# fmt: on

data_agent_context = geminidataanalytics.DataAgentContext(
    data_agent=data_chat_client.data_agent_path(
        billing_project, location, data_agent_id
    ),
    # credentials=credentials  # Uncomment this if you're using Looker datasource
)


# Helper function for calling the API
def multi_turn_conversation(msg):
    message = geminidataanalytics.Message(
        user_message=geminidataanalytics.UserMessage(text=msg)
    )

    # Send multi-turn request by including previous turns plus new message
    conversation_messages.append(message)

    request = geminidataanalytics.ChatRequest(
        parent=f"projects/{billing_project}/locations/{location}",
        messages=conversation_messages,
        data_agent_context=data_agent_context,
        # uncomment the line below when using chat with inline_context
        # inline_context=inline_context
    )

    # Make the request
    stream = data_chat_client.chat(request=request)

    # Handle the response
    for response in stream:
        show_message(response)
        if response.system_message or response.user_message:
            conversation_messages.append(response)

In [ ]:
# Send first-turn request
multi_turn_conversation("I wanna analyze how we can build a good stage so that we can keep the users to play our title. What should we do for analzying it to pull out insights and what data we should see?")

In [ ]:
# Send follow-up-turn request
multi_turn_conversation("사용자 참여, 스테이지별 평균 플레이 시간과 스테이지별 플레이 횟수, 완료율의 지표를 확인할 것, 그리고 진행분석 퍼널 분석을 수행할 것, 스테이지별 유지율을 살펴본 후, 어떤 스테이지가 인기가 있는지에 대해서 알려줘. 그리고 그 분석 과정에 대해서도 단계적인 사고의 흐름과 어떠한 논리로 각각의 지표들을 구했는지에 대해서도 설명해줘.")

# Other DataAgent Service Methods

In [ ]:
# @title Get Data Agent

# Initialize request argument(s)

# fmt: off
data_agent_id = "data_agent_1"  # @param {type:"string"}
# fmt: on
request = geminidataanalytics.GetDataAgentRequest(
    name=data_agent_client.data_agent_path(billing_project, location, data_agent_id)
)

# Make the request
response = data_agent_client.get_data_agent(request=request)

# Handle the response
print(response)

In [ ]:
# @title List Data Agents

# Initialize request argument(s)
request = geminidataanalytics.ListDataAgentsRequest(
    parent=f"projects/{billing_project}/locations/{location}",
)

# Make the request
page_result = data_agent_client.list_data_agents(request=request)

# Handle the response
for response in page_result:
    print(response)

In [ ]:
# @title List Accessible Data Agents

# fmt: off
creator_filter = "NONE"  # @param ["NONE", "CREATOR_ONLY", "NOT_CREATOR_ONLY"]
# fmt: on

# Initialize request argument(s)
request = geminidataanalytics.ListAccessibleDataAgentsRequest(
    parent=f"projects/{billing_project}/locations/{location}",
    creator_filter=creator_filter,
)

# Make the request
page_result = data_agent_client.list_accessible_data_agents(request=request)

# Handle the response
for response in page_result:
    print(response)

In [ ]:
# @title Update Data Agent

# Initialize request argument(s)

# fmt: off
data_agent_id = "data_agent_1"  # @param {type:"string"}
# fmt: on

data_agent = geminidataanalytics.DataAgent(
    data_analytics_agent=geminidataanalytics.DataAnalyticsAgent(
        published_context=published_context
    ),
    name=data_agent_client.data_agent_path(billing_project, location, data_agent_id),
    description="This is my new updated description.",
)


update_mask = field_mask_pb2.FieldMask(
    paths=["description", "data_analytics_agent.published_context"]
)

request = geminidataanalytics.UpdateDataAgentRequest(
    data_agent=data_agent,
    update_mask=update_mask,
)

try:
    operation = data_agent_client.update_data_agent(request=request)

    while not operation.done():
        print("Still in progress...")
        time.sleep(2)

    if operation.done():
        if operation.exception():
            print("Operation failed:", operation.exception())
        else:
            print("Operation succeeded:", operation.result())
except Exception as e:
    print(f"Error updating Data Agent: {e}")

In [ ]:
# @title [Agent Sharing] Set IAM Policy for Data Agent

""" PLEASE NOTE THIS API CALLS OVERRIDES EXISTING PERMISSION FOR THE RESOURCE.
For preserving existing policy in practise call Get IAM policy to fetch existing policy and pass it along with additional changes
in the call to Set IAM Policy
"""

# Inputs
# fmt: off
data_agent_id = "data_agent_1"  # @param {type:"string"}
role = "roles/geminidataanalytics.dataAgentEditor"  # @param {type:"string"}
users = "abc@google.com, wxyz@google.com"  # @param {type:"string"}
# fmt: on

resource = f"projects/{billing_project}/locations/global/dataAgents/{data_agent_id}"

# Construct the IAM policy
binding = policy_pb2.Binding(
    role=role, members=[f"user:{i.strip()}" for i in users.split(",")]
)

policy = policy_pb2.Policy(bindings=[binding])

# Create the request
request = iam_policy_pb2.SetIamPolicyRequest(resource=resource, policy=policy)

# Send the request
try:
    response = data_agent_client.set_iam_policy(request=request)
    print("IAM Policy set successfully!")
    print(f"Response: {response}")
except Exception as e:
    print(f"Error setting IAM policy: {e}")

In [ ]:
# @title [Agent Sharing] GET IAM Policy for Data Agent

# Initialize request argument(s)

# fmt: off
data_agent_id = "data_agent_1"  # @param {type:"string"}
# fmt: on

resource = f"projects/{billing_project}/locations/global/dataAgents/{data_agent_id}"
request = iam_policy_pb2.GetIamPolicyRequest(
    resource=resource,
)
try:
    response = data_agent_client.get_iam_policy(request=request)
    print("IAM Policy fetched successfully!")
    print(f"Response: {response}")
except Exception as e:
    print(f"Error setting IAM policy: {e}")

In [ ]:
# @title [Soft Delete] Delete Data Agent

# Initialize request argument(s)

# fmt: off
data_agent_id = "data_agent_1"  # @param {type:"string"}
# fmt: on

request = geminidataanalytics.DeleteDataAgentRequest(
    name=data_agent_client.data_agent_path(billing_project, location, data_agent_id)
)

try:
    # Make the request
    data_agent_client.delete_data_agent(request=request)
    print("Data Agent Deleted")
except Exception as e:
    print(f"Error deleting Data Agent: {e}")

# Other DataChat Service Methods

In [ ]:
# @title Get Conversation

# fmt: off
conversation_id = "conversation_1"  # @param {type:"string"}
# fmt: on

request = geminidataanalytics.GetConversationRequest(
    name=data_chat_client.conversation_path(billing_project, location, conversation_id),
)

# Make the request
response = data_chat_client.get_conversation(request=request)

# Handle the response
print(response)

In [ ]:
# @title List Conversation

request = geminidataanalytics.ListConversationsRequest(
    parent=f"projects/{billing_project}/locations/{location}",
)

# Make the request
response = data_chat_client.list_conversations(request=request)

# Handle the response
print(response)

In [ ]:
# @title List Messages

# fmt: off
conversation_id = "conversation_1"  # @param {type:"string"}
# fmt: on

request = geminidataanalytics.ListMessagesRequest(
    parent=data_chat_client.conversation_path(
        billing_project, location, conversation_id
    ),
)

# Make the request
response = data_chat_client.list_messages(request=request)

# Handle the response
print(response)

In [ ]:
# @title Get Conversation

# fmt: off
conversation_id = "conversation_1"  # @param {type:"string"}
# fmt: on

request = geminidataanalytics.DeleteConversationRequest(
    name=data_chat_client.conversation_path(billing_project, location, conversation_id),
)

# Make the request
response = data_chat_client.delete_conversation(request=request)

# Handle the response
print(response)